In [1]:
# %pip install google-api-python-client
# %pip install nltk
# %pip install contractions

In [2]:
import googleapiclient.discovery
import googleapiclient.errors

import pandas as pd
import psycopg2

#### Extract

In [3]:
import googleapiclient.discovery

# Set up API credentials
api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = "****-YTY"

# Initialize the YouTube API client
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY
)

# Function to fetch all comments
def fetch_all_comments(video_id):
    comments = []
    next_page_token = None

    while True:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,  # Maximum allowed value
            pageToken=next_page_token
        )
        response = request.execute()

        # Extract comments from the response
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append([
                comment['publishedAt'],
                comment['likeCount'],
                comment['textDisplay']
            ])

        # Check if there's another page
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        

    return comments

# Fetch comments for a video
video_id = "dtp6b76pMak"  
all_comments = fetch_all_comments(video_id)
print(f"Total Comments Fetched: {len(all_comments)}")


Total Comments Fetched: 27305


In [ ]:
# all_comments

In [5]:
df = pd.DataFrame(all_comments, columns=['dated', 'likes', 'text'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27305 entries, 0 to 27304
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dated   27305 non-null  object
 1   likes   27305 non-null  int64 
 2   text    27305 non-null  object
dtypes: int64(1), object(2)
memory usage: 640.1+ KB


#### Transform

In [7]:
# Cleaning libraries

import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import contractions

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dooterior/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/dooterior/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
# Remove HTML tags
def remove_html_tags(text):
    html_tag_pattern = re.compile(r'<a href=".*?">.*?</a>')  
    return html_tag_pattern.sub('', text)

# Apply the function to the DataFrame
df['text'] = df['text'].apply(remove_html_tags)
df.head()

,dated,likes,text
0,2024-12-27T21:33:28Z,1,You explain things SUPER WELL!!!!!
1,2024-12-27T00:41:30Z,1,Essentially you are using 2 computers at once...
2,2024-12-27T00:36:08Z,0,I would describe as just the beginning or ear...
3,2024-12-27T00:33:23Z,0,"yo😮, I would need to buy new jeans and a turt..."
4,2024-12-27T00:30:20Z,0,You are honestly underselling the actual expe...


In [9]:
# convert the date column to datetime format
df['dated'] = pd.to_datetime(df['dated'])

# change the datetime format
df['date'] = df['dated'].dt.strftime('%Y-%m-%d')

In [10]:
# Download stopwords
nltk.download('stopwords')
nltk.download('punkt')

# Initialize stopwords list
stop_words = set(stopwords.words('english'))

# Function to clean text
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    #fix contracted words
    text = contractions.fix(text)
    
    # Remove mentions, hashtags, URLs
    text = re.sub(r'@\w+|#\w+|http\S+', '', text)
    
    # Remove special characters and numbers
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Tokenization
    tokens = word_tokenize(text, preserve_line=True)
    
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    
    # Join tokens back to a single string
    clean_text = ' '.join(tokens)
    
    return clean_text

# Apply the cleaning function to your text column
df['cleaned_text'] = df['text'].apply(clean_text)

# View the cleaned data

df[['text', 'cleaned_text']].head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dooterior/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/dooterior/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,text,cleaned_text
0,You explain things SUPER WELL!!!!!,explain things super well
1,Essentially you are using 2 computers at once...,essentially using computers work together big ...
2,I would describe as just the beginning or ear...,would describe beginning early public rnd cons...
3,"yo😮, I would need to buy new jeans and a turt...",yo would need buy new jeans turtleneck sick
4,You are honestly underselling the actual expe...,honestly underselling actual experience


In [11]:
df.head()

,dated,likes,text,date,cleaned_text
0,2024-12-27 21:33:28+00:00,1,You explain things SUPER WELL!!!!!,2024-12-27,explain things super well
1,2024-12-27 00:41:30+00:00,1,Essentially you are using 2 computers at once...,2024-12-27,essentially using computers work together big ...
2,2024-12-27 00:36:08+00:00,0,I would describe as just the beginning or ear...,2024-12-27,would describe beginning early public rnd cons...
3,2024-12-27 00:33:23+00:00,0,"yo😮, I would need to buy new jeans and a turt...",2024-12-27,yo would need buy new jeans turtleneck sick
4,2024-12-27 00:30:20+00:00,0,You are honestly underselling the actual expe...,2024-12-27,honestly underselling actual experience


#### Load

In [14]:
conn = psycopg2.connect(
    dbname="api_data", user="postgres", password="1234!", host="localhost", port="5432"
)
cur = conn.cursor()

for _, row in df.iterrows():
    cur.execute(
        "INSERT INTO vision_pro_reviews (date, review) VALUES (%s, %s)",
        (row['date'], row['cleaned_text'])
    )
print('successfully loaded')
conn.commit()
cur.close()
conn.close()

successfully loaded


In [13]:
# df = pd.read_sql_query("SELECT * FROM yt_feedback", conn)

#### Automate

In [ ]:
# crontab -e

# 0 0 * * * 0 0 * * * /usr/bin/python3 /path/to/ytpipeline.py

#:wq